# Velocity Control Kinematic Simulation

### Requirements
Conda
- cmake
- pybind11

Pip
- numpy
- sophuspy
- matplotlib

In [1]:
import numpy as np
import sophus as sp
import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

In [109]:
# ----- HYPERPARAMS
# Sim
SPEEDUP = 4

TIME_HORIZON = 12
SIM_HZ = 240
CONTROL_HZ = 20
VIZ_HZ = 24 / SPEEDUP

# Viz
R_ROBOT = 0.2
BUTT_SAMPLES = 20
X_LIM = [-2, 2]
Y_LIM = [-2, 2]

In [110]:
# ----- SIMULATION ENV
class Env:
    def __init__(self, hz):
        self.pos_state = np.zeros(3)
        self.vel_state = np.zeros(2)
        self.dt = 1.0 / hz
        
    def get_pose(self):
        return self.pos_state
    
    def step(self, vel_input=None):
        if vel_input is not None:
            self.vel_state = vel_input
            
        self.pos_state[0] += self.vel_state[0] * np.cos(self.pos_state[2]) * self.dt
        self.pos_state[1] += self.vel_state[0] * np.sin(self.pos_state[2]) * self.dt
        self.pos_state[2] += self.vel_state[1] * self.dt

In [111]:
# ----- VISUALIZER
class Viz:
    def __init__(self):
        self.frames = []
        
        # Robot local vertices
        xy_ul = np.array([R_ROBOT / 2, R_ROBOT])
        xy_ur = np.array([R_ROBOT / 2, -R_ROBOT])
        xy_l = np.array([-R_ROBOT / 2, R_ROBOT])
        xy_r = np.array([-R_ROBOT / 2, -R_ROBOT])
        
        xy_list = [xy_r, xy_ur, xy_ul, xy_l]
        for phi in np.linspace(np.pi / 2, 3 * np.pi / 2, BUTT_SAMPLES):
            xy_list.append(np.array([R_ROBOT * (np.cos(phi) - 0.5), R_ROBOT * np.sin(phi)]))
            
        self.xy_verts_local = np.array(xy_list)
        
    def _draw_robot(self, xyt, color):
        theta = xyt[2]
        ct = np.cos(theta)
        st = np.sin(theta)
        trans = np.array([[ct, -st], [st, ct]])
        
        xy_verts_transformed = (trans @ self.xy_verts_local.T).T + xyt[None, :2]
        
        self.ax.plot(xyt[0], xyt[1], color=color, marker='o')
        self.ax.plot(xy_verts_transformed[:, 0], xy_verts_transformed[:, 1], color=color)
        
    def _render_frame(self, frame):
        plt.cla()
        
        self.ax.set_xlim(*X_LIM)
        self.ax.set_ylim(*Y_LIM)
        self.ax.set_aspect("equal")
        
        self._draw_robot(frame[1], '0.7')
        self._draw_robot(frame[0], '0')
        
    def clear_frames(self):
        self.frames = []
    
    def record_frame(self, xyt_robot, xyt_goal):
        self.frames.append((xyt_robot, xyt_goal))
        
    def animate(self, playback_speed=1.0):  
        plt.ioff()
        fig, self.ax = plt.subplots()
        
        return animation.FuncAnimation(fig, self._render_frame, frames=self.frames, interval=1000 / VIZ_HZ / playback_speed, repeat=False)
    
    
    

In [112]:
# ----- CONTROLLER
from scipy.spatial.transform import Rotation

V_MAX_DEFAULT = 0.2  # base.params["motion"]["default"]["vel_m"]
W_MAX_DEFAULT = 0.45  # (vel_m_max - vel_m_default) / wheel_separation_m
ACC_LIN = 0.2  # 0.5 * base.params["motion"]["max"]["accel_m"]
ACC_ANG = 0.6  # 0.5 * (accel_m_max - accel_m_default) / wheel_separation_m
MAX_HEADING_ANG = np.pi / 4

def xyt2sp(xyt):
    r_mat = sp.SO3.exp(np.array([0, 0, xyt[2]])).matrix()
    t = np.array([xyt[0], xyt[1], 0.])
    return sp.SE3(r_mat, t)

def sp2xyt(pose):
    return [pose.translation()[0], pose.translation()[1], pose.so3().log()[2]]

def transform_global_to_base(xyt_target, xyt_base):
    pose_target = xyt2sp(xyt_target)
    pose_base = xyt2sp(xyt_base)
    pose_rel = pose_base.inverse() * pose_target
    return sp2xyt(pose_rel)

class Controller:
    def __init__(self, track_yaw=True):
        self.track_yaw = track_yaw
        
        # Params
        self.v_max = V_MAX_DEFAULT
        self.w_max = W_MAX_DEFAULT
        self.lin_error_tol = self.v_max / CONTROL_HZ
        self.ang_error_tol = self.w_max / CONTROL_HZ
        
        # Init
        self.xyt_goal = np.zeros(3)
        self.dxyt_goal = np.zeros(3)
    
    def set_goal(self, goal, vel_goal=None):
        self.xyt_goal = goal
        if vel_goal is not None:
            self.dxyt_goal = vel_goal
    
    def _compute_error_pose(self, xyt_base):
        """
        Updates error based on robot localization
        """
        xyt_err = transform_global_to_base(self.xyt_goal, xyt_base)
        if not self.track_yaw:
            xyt_err[2] = 0.0

        return xyt_err
    
    @staticmethod
    def _velocity_feedback_control(x_err, a, v_max):
        """
        Computes velocity based on distance from target.
        Used for both linear and angular motion.

        Current implementation: Trapezoidal velocity profile
        """
        t = np.sqrt(2.0 * abs(x_err) / a)  # x_err = (1/2) * a * t^2
        v = min(a * t, v_max)
        return v * np.sign(x_err)
        
    @staticmethod
    def _turn_rate_limit(lin_err, heading_diff, w_max, tol=0.0):
        """
        Compute velocity limit that prevents path from overshooting goal
        
        heading error decrease rate > linear error decrease rate
        (w - v * np.sin(phi) / D) / phi > v * np.cos(phi) / D
        v < (w / phi) / (np.sin(phi) / D / phi + np.cos(phi) / D)
        v < w * D / (np.sin(phi) + phi * np.cos(phi))
        
        (D = linear error, phi = angular error)
        """
        assert lin_err >= 0.0
        assert heading_diff >= 0.0
        
        if heading_diff > MAX_HEADING_ANG:
            return 0.0
        else:
            return w_max * lin_err / (np.sin(heading_diff) + heading_diff * np.cos(heading_diff) + 1e-5)
    
    def _feedback_traj_track(self, xyt_err):
        xyt_err = self._compute_error_pose(xyt)
        v_raw = V_MAX_DEFAULT * (K1 * xyt_err[0] + xyt_err[1] * np.tan(xyt_err[2])) / np.cos(xyt_err[2])
        w_raw = V_MAX_DEFAULT * (K2 * xyt_err[1] + K3 * np.tan(xyt_err[2])) / np.cos(xyt_err[2])**2
        v_out = min(v_raw, V_MAX_DEFAULT)
        w_out = min(w_raw, W_MAX_DEFAULT)
        return np.array([v_out, w_out])
    
    def _feedback_simple(self, xyt_err):
        v_cmd = w_cmd = 0

        lin_err_abs = np.linalg.norm(xyt_err[0:2])
        ang_err = xyt_err[2]

        # Go to goal XY position if not there yet
        if lin_err_abs > self.lin_error_tol:
            heading_err = np.arctan2(xyt_err[1], xyt_err[0])
            heading_err_abs = abs(heading_err)

            # Compute linear velocity
            v_raw = self._velocity_feedback_control(
                lin_err_abs, ACC_LIN, self.v_max
            )
            v_limit = self._turn_rate_limit(
                lin_err_abs,
                heading_err_abs,
                self.w_max / 2.0,
                tol=self.lin_error_tol,
            )
            v_cmd = np.clip(v_raw, 0.0, v_limit)

            # Compute angular velocity
            w_cmd = self._velocity_feedback_control(
                heading_err, ACC_ANG, self.w_max
            )

        # Rotate to correct yaw if yaw tracking is on and XY position is at goal
        elif abs(ang_err) > self.ang_error_tol and self.track_yaw:
            # Compute angular velocity
            w_cmd = self._velocity_feedback_control(
                ang_err, ACC_ANG, self.w_max
            )
            
        return v_cmd, w_cmd

    def forward(self, xyt):
        xyt_err = self._compute_error_pose(xyt)
        return self._feedback_simple(xyt_err)
        
    

In [113]:
# ----- GOAL
GOAL = [0.2, -0.5, -0.3]

In [114]:
# ----- MAIN
# initialize objects
xyt_goal = np.array(GOAL)

viz = Viz()
env = Env(SIM_HZ)
agent = Controller()
agent.set_goal(xyt_goal)

# simulate
t_control = 0
t_viz = 0
dt_control = 1 / CONTROL_HZ
dt_viz = 1 / VIZ_HZ
for t in np.linspace(0, TIME_HORIZON, int(TIME_HORIZON * SIM_HZ)):
    xyt = env.get_pose()
    
    vel_command = None
    if t >= t_control:
        vel_command = agent.forward(xyt)
        t_control += dt_control
    if t >= t_viz:
        viz.record_frame(xyt.copy(), xyt_goal.copy())
        t_viz += dt_viz
    
    env.step(vel_command)
    
# animate
viz.animate(playback_speed=SPEEDUP)